# 1. 自动识别

In [1]:
import os

In [2]:
list_file = list(os.walk('./'))[0][2]
if '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
if '0.null.txt' in list_file:
    list_file.remove('0.null.txt')
    
for file in list_file:
    if '整理好的替代料总表' in file:
        alternate = './' + file
    elif '改价表' in file:
        stock = './' + file
    elif 'MRP' in file:
        mrp = './' + file

print('alternate:     ' + alternate)
print('stock:         ' + stock)
print('mrp:           ' + mrp)

alternate:     ./整理好的替代料总表_20240320_203811.xlsx
stock:         ./改价表 3-19.xlsx
mrp:           ./US MRP20240318-运营.xlsx


# 2. 库存计算

In [3]:
import pandas as pd

In [4]:
df_alternate = pd.read_excel(alternate,
                             header=0,
                             dtype=str).fillna('')

df_alternate['row'] = [i+1 for i in range(len(df_alternate))]

df_alternate

,主料SKU,替换料1,替换料2,替换料3,替换料4,替换料5,替换料6,row
0,USABS00021-C,USABS00021,,,,,,1
1,USABS00117B-C,USABS00117B,USABS00117,,,,,2
2,USABS00217B-C,USABS00217B,USABS00217,,,,,3
3,USABS00317B-C,USABS00317B,USABS00317,,,,,4
4,USABS00417-C,USABS00417,,,,,,5
...,...,...,...,...,...,...,...,...
48787,USWT80460,USET80460,,,,,,48788
48788,USWT90820,USET90820,,,,,,48789
48789,USWT91020,USET91020,,,,,,48790
48790,USWT91520,USET91520,,,,,,48791


In [5]:
list_df = []
for column in df_alternate.columns.tolist()[:-1]:
    df_temp =  df_alternate[['主料SKU', column, 'row']]
    df_temp.columns = ['主料', 'SKU', 'row']
    df_temp = df_temp[df_temp['SKU'] != '']
    list_df.append(df_temp)

df_alternate = pd.concat(list_df, ignore_index=True)

df_alternate

,主料,SKU,row
0,USABS00021-C,USABS00021-C,1
1,USABS00117B-C,USABS00117B-C,2
2,USABS00217B-C,USABS00217B-C,3
3,USABS00317B-C,USABS00317B-C,4
4,USABS00417-C,USABS00417-C,5
...,...,...,...
106196,USGS6310TNZB-C,USGS6310TNZ,28299
106197,USIC02680L-6-C,USIC02680-6,31066
106198,USIC02680L-C,USIC02680,31067
106199,USIC2A366Z-C,USIC2A366,31174


In [6]:
df_stock = pd.read_excel(stock,
                         sheet_name=0,
                         header=0,
                         names=['SKU', '库存'],
                         usecols='A, F',
                         dtype=str).fillna('')

df_stock['SKU'] = ['' if sku.strip().endswith('-NC') else sku.strip()[:-2] if sku.strip().endswith('-N') else sku.strip() for sku in df_stock['SKU'].tolist()]
df_stock = df_stock[df_stock['SKU'] != ''].reset_index(drop=True)
df_stock['库存'] = [int(stock.strip()) for stock in df_stock['库存'].tolist()]
df_stock = df_stock.groupby([df_stock['SKU']], as_index=False).sum()

df_stock

,SKU,库存
0,BCB96454-XH,0
1,BPD14040-XH,0
2,BPD8550086500+BRS31318,1
3,BRR34279-XH,0
4,CABM0021A,0
...,...,...
106467,YHTWS58202-Y-6,0
106468,YHTWS5W1ZBA-Y-10,0
106469,YHTWS5W1ZBA-Y-8,0
106470,YHTWS802AA-Y-6,0


In [7]:
print(df_alternate.columns)
print(df_stock.columns)

df_temp = pd.merge(left=df_alternate,
                   right=df_stock,
                   how='left',
                   left_on='SKU',
                   right_on='SKU').fillna(0)


df_temp

Index(['主料', 'SKU', 'row'], dtype='object')
Index(['SKU', '库存'], dtype='object')


,主料,SKU,row,库存
0,USABS00021-C,USABS00021-C,1,7.0
1,USABS00117B-C,USABS00117B-C,2,5.0
2,USABS00217B-C,USABS00217B-C,3,1.0
3,USABS00317B-C,USABS00317B-C,4,25.0
4,USABS00417-C,USABS00417-C,5,0.0
...,...,...,...,...
106196,USGS6310TNZB-C,USGS6310TNZ,28299,0.0
106197,USIC02680L-6-C,USIC02680-6,31066,0.0
106198,USIC02680L-C,USIC02680,31067,0.0
106199,USIC2A366Z-C,USIC2A366,31174,0.0


In [8]:
df_temp = df_temp[['row', '库存']]
df_temp = df_temp.groupby([df_temp['row']], as_index=False).sum()

df_temp

,row,库存
0,1,8.0
1,2,5.0
2,3,9.0
3,4,36.0
4,5,5.0
...,...,...
48787,48788,6.0
48788,48789,26.0
48789,48790,9.0
48790,48791,3.0


In [9]:
print(df_alternate.columns)
print(df_temp.columns)

df_alternate = pd.merge(left=df_alternate,
                        right=df_temp,
                        how='left',
                        left_on='row',
                        right_on='row').fillna(0)

df_alternate

Index(['主料', 'SKU', 'row'], dtype='object')
Index(['row', '库存'], dtype='object')


,主料,SKU,row,库存
0,USABS00021-C,USABS00021-C,1,8.0
1,USABS00117B-C,USABS00117B-C,2,5.0
2,USABS00217B-C,USABS00217B-C,3,9.0
3,USABS00317B-C,USABS00317B-C,4,36.0
4,USABS00417-C,USABS00417-C,5,5.0
...,...,...,...,...
106196,USGS6310TNZB-C,USGS6310TNZ,28299,22.0
106197,USIC02680L-6-C,USIC02680-6,31066,68.0
106198,USIC02680L-C,USIC02680,31067,2.0
106199,USIC2A366Z-C,USIC2A366,31174,13.0


In [10]:
print(df_stock.columns)
print(df_alternate.columns)

df_temp = pd.merge(left=df_stock,
                   right=df_alternate,
                   how='left',
                   left_on='SKU',
                   right_on='SKU',
                   suffixes=('', '_alternate')).fillna('')

df_temp = df_temp[df_temp['row'] == '']
df_temp['主料'] = df_temp['SKU'].tolist()
df_temp = df_temp[['主料', 'SKU', 'row', '库存']]

df_temp

Index(['SKU', '库存'], dtype='object')
Index(['主料', 'SKU', 'row', '库存'], dtype='object')


,主料,SKU,row,库存
0,BCB96454-XH,BCB96454-XH,,0
1,BPD14040-XH,BPD14040-XH,,0
2,BPD8550086500+BRS31318,BPD8550086500+BRS31318,,1
3,BRR34279-XH,BRR34279-XH,,0
4,CABM0021A,CABM0021A,,0
...,...,...,...,...
106467,YHTWS58202-Y-6,YHTWS58202-Y-6,,0
106468,YHTWS5W1ZBA-Y-10,YHTWS5W1ZBA-Y-10,,0
106469,YHTWS5W1ZBA-Y-8,YHTWS5W1ZBA-Y-8,,0
106470,YHTWS802AA-Y-6,YHTWS802AA-Y-6,,0


In [11]:
df_stock = pd.concat([df_alternate, df_temp], ignore_index=True)
df_stock['new_row'] = [i+1 for i in range(len(df_stock))]
df_stock['row'] = [row if row != '' else new_row for row, new_row in zip(df_stock['row'].tolist(), df_stock['new_row'].tolist())]
df_stock = df_stock[['主料', 'SKU', 'row', '库存']]

df_stock

,主料,SKU,row,库存
0,USABS00021-C,USABS00021-C,1,8.0
1,USABS00117B-C,USABS00117B-C,2,5.0
2,USABS00217B-C,USABS00217B-C,3,9.0
3,USABS00317B-C,USABS00317B-C,4,36.0
4,USABS00417-C,USABS00417-C,5,5.0
...,...,...,...,...
133511,YHTWS58202-Y-6,YHTWS58202-Y-6,133512,0.0
133512,YHTWS5W1ZBA-Y-10,YHTWS5W1ZBA-Y-10,133513,0.0
133513,YHTWS5W1ZBA-Y-8,YHTWS5W1ZBA-Y-8,133514,0.0
133514,YHTWS802AA-Y-6,YHTWS802AA-Y-6,133515,0.0


# 3. 库存生成

In [12]:
df_mrp = pd.read_excel(mrp,
                       header=1,
                       usecols='B',
                       dtype=str).fillna('')

df_mrp

,MRP SKU
0,USOBDAD100-C
1,USTCV17215HIE-2-C
2,USCAK14152U-C
3,USOBDTK100
4,USWS02632U-C
...,...
68657,USWHN11008
68658,USWHN11288
68659,USWHN11236
68660,USWHS10547


In [13]:
import datetime

In [14]:
print(df_mrp.columns)
print(df_stock.columns)

df_stock = pd.merge(left=df_mrp,
                    right=df_stock[['SKU', '库存']],
                    how='left',
                    left_on='MRP SKU',
                    right_on='SKU').fillna(0)

df_stock[['MRP SKU', '库存']].to_excel('./整理好的库存表_' + datetime.datetime.now().strftime("%Y%m%d_%H%M%S") + '.xlsx', index=False)

df_stock

Index(['MRP SKU'], dtype='object')
Index(['主料', 'SKU', 'row', '库存'], dtype='object')


,MRP SKU,SKU,库存
0,USOBDAD100-C,USOBDAD100-C,173.0
1,USTCV17215HIE-2-C,USTCV17215HIE-2-C,603.0
2,USCAK14152U-C,USCAK14152U-C,142.0
3,USOBDTK100,USOBDTK100,261.0
4,USWS02632U-C,USWS02632U-C,401.0
...,...,...,...
68657,USWHN11008,0,0.0
68658,USWHN11288,USWHN11288,0.0
68659,USWHN11236,USWHN11236,0.0
68660,USWHS10547,0,0.0
